In [40]:
from common import *
%matplotlib inline
canvas(9, 6)

In [2]:
from sklearn.linear_model import LinearRegression as LR

def do_lr(X, Y, W):
    lr_list = []
    # assert the columns of the input can be indexed by range(0, 3023)
    for k in Y.columns:
        x, y, w = X[k], Y[k], W[k]
        if w.sum() == 0:
            lr_list += [None]
            continue
        lr = LR(fit_intercept=False)
        # don't forget x1 and x2 have NaNs for valid y. try setting W=0 for when y and W does not exist and then filling in NaNs
        lr = lr.fit(x[:, None], y, w)
        lr_list += [lr]
    return lr_list

In [54]:
ifilt = index_nvdf(dh, 246)
sfilt = stock_nvdf(dh, 246)
Xpos = make_stock_wide(dh.x3 + dh.x4 - dh.x5).pct().fillna(0.5) * 2 - 1
Xneg = make_stock_wide(dh.x3 + dh.x4 + dh.x5).pct().fillna(0.5) * 2 - 1
Y = make_stock_wide(dh.y).fillna(0)
W = make_stock_wide(dh.Weight).fillna(0)

In [55]:
tr = Y.index.isin(df.Day)
cv = Y.index.isin(dg.Day)

In [111]:
sub = sfilt # <===============
X = Xpos # <==============

In [112]:
Xtr = X[tr].T[sub].T
Ytr = Y[tr].T[sub].T
Wtr = W[tr].T[sub].T

In [113]:
lr_list = do_lr(Xtr, Ytr - Ytr.mean(), Wtr)

In [114]:
st = tr # <===============
Xcv = X[st].T[sub].T
Ycv = Y[st].T[sub].T
Wcv = W[st].T[sub].T

In [115]:
Ycv_ = np.stack([(lr.predict(Xcv[k][:, None]) if lr is not None else np.zeros(Ycv.shape[0]))
               for lr, k in zip(lr_list, Ycv.columns)], axis=1)
Ycv_ = Ycv_ + Ycv * 0 + Ytr.mean()

In [116]:
((Ycv_ - Ycv) ** 2 * Wcv).sum().sum() / ((Ycv ** 2 * Wcv).sum().sum())

0.7106209884352731

### stuff's going down

In [101]:
Ypos = Ycv_

In [43]:
def get_ans(i):
    row = dh.loc[i]
    which = filt.loc[i]
    return ypos.loc[row.Day, row.Stock] if which else yneg.loc[row.Day, row.Stock]
yy = pd.Series(np.array([get_ans(i) for i in dh.index]), index=dh.index)

KeyboardInterrupt: 